In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-infobars")
# options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_experimental_option("prefs", { \
    "profile.default_content_setting_values.media_stream_mic": 2,     # 1:allow, 2:block 
    "profile.default_content_setting_values.media_stream_camera": 2,  # 1:allow, 2:block 
    "profile.default_content_setting_values.geolocation": 2,          # 1:allow, 2:block 
    "profile.default_content_setting_values.notifications": 2         # 1:allow, 2:block 
  })

import pandas as pd

In [2]:
# declare global variables

WAIT_TIME = 10 # seconds

# default input & output file names
input_file = 'rico_products.csv'
output_file = 'rico_output.csv'
url = 'http://b2b.daddariocanada.com/#/login'

username = ''
password = ''

In [3]:
# create empty dataframe
df = pd.DataFrame(columns=[ 'Product Model', 'Product Code', 'Product Cost'])

product_df = pd.read_csv(input_file)
product_list = product_df['Variant SKU'].tolist()

In [4]:
print(f'Scraping data for {len(product_list)} products:')
for product in product_list:
    print(f'\t{product}')

Scraping data for 138 products:
	RCA2515
	RCA2520
	RCA2525
	RCA2530
	RCA2535
	RJA2515
	RJA2520
	RJA2525
	RJA2530
	RJA2535
	RCA1015
	RCA1020
	RCA1025
	RCA1030
	RCA1035
	RCA1040
	RCA0315
	RCA0320
	RCA0325
	RCA0330
	RCA0335
	RJB1015
	RJB1020
	RJB1025
	RJB1030
	RJB1035
	RJB1040
	RJB0330
	RJB0325
	RJB0320
	RJB0315
	RDB1015
	RDB1020
	RDB1025
	RDB1030
	RDB1035
	RDB1040
	RJC10SF
	RJC10MS
	RJC10MD
	RJC10MH
	RJC10HD
	RSF10ASX2S
	RSF10ASX2M
	RSF10ASX2H
	RSF10ASX3S
	RSF10ASX3M
	RSF10ASX3H
	RSF10ASX4S
	RSF10ASX4M
	RSF10ASX4M
	RSF10ASX2S
	RSF10ASX2M
	RSF10ASX2H
	RSF10ASX3S
	RSF10ASX3M
	RSF10ASX3H
	RSF10ASX4S
	RSF10ASX4M
	RSF10ASX4H
	RSF10SSX4M
	RRS10SSX4M
	RSF10SSX4H
	RRS10SSX4H
	RSF10SSX4S
	RRS10SSX4S
	RSF10SSX3M
	RRS10SSX3M
	RSF10SSX3H
	RRS10SSX3M
	RSF10SSX3S
	RRS10SSX3S
	RSF10SSX2M
	RRS10SSX2M
	RSF10SSX2H
	RRS10SSX2H
	RSF10SSX2S
	RRS10SSX2S
	RRGMPCASXA3
	RRGMPCASXA5
	RRGMPCASXA7
	RRGMPCASXB3
	RRGMPCASXB5
	RRGMPCASXB7
	RRGMPCASXC3
	RRGMPCASXC5
	RRGMPCASXC7
	RRGMPCBCLA3
	RRGMPCBCLA5
	RRGMPCBCLA7
	R

In [5]:
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
print(f'Opening {url} in Chrome browser...')
driver.get(url)
products_not_found = 0
products_not_found_list = []

C:\Users\AYNM\AppData\Local\Temp\ipykernel_20136\600678085.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
C:\Users\AYNM\AppData\Local\Temp\ipykernel_20136\600678085.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


Opening http://b2b.daddariocanada.com/#/login in Chrome browser...


In [6]:
# enter username
username_input = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="username"]')))
username_input.clear()
username_input.send_keys(username)


In [7]:
# enter password
password_input = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="password"]')))
password_input.clear()
password_input.send_keys(password)
    

In [8]:
# click login button
login_button = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="login-signin"]'))).click()

In [11]:
for product_model in product_list:
# enter product model in search box

    try:
        search_input = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[name="catalogSearchInput"]')))
        search_input.clear()
        search_input.send_keys(product_model)
        search_input.send_keys(Keys.RETURN)
    except:
        print('Search input not found')

        

    # get dealer price
    try:
        # product_link = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(), '{product_model}')]"))).click()
        dealer_price = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[class="ng-binding red"]'))).text

    except:
        print(f'Product not found: {product_model}')
        products_not_found += 1
        products_not_found_list.append(product_model)
        continue

    # get list price

    try:
        list_price = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[class="ng-binding strike"]'))).text
    except:
        print(f'Product list price not found: {product_model}')

    
    # get product code (to make sure we have right price)
    try:
        product_code = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.XPATH, f"//span[text()='{product_model}']"))).text
    except:
        print(f'Product code not found: {product_model}')

    print(f'model: {product_model}, product code: {product_code}')
    print(f'dealer price: {dealer_price}')
    print(f'list price: {list_price}')
    print('\n')
    
    # add row to dataframe
    row = {'Product Model': product_model, 'Product Code': product_code, 'Product Cost': dealer_price, 'Product List Price': list_price}
    df_new_row = pd.DataFrame([row])
    df = pd.concat([df, df_new_row], axis=0, ignore_index=True)



model: RCA2515, product code: RCA2515
dealer price: 39.45
list price: 80.50


model: RCA2520, product code: RCA2520
dealer price: 39.45
list price: 80.50


model: RCA2525, product code: RCA2525
dealer price: 39.45
list price: 80.50


model: RCA2530, product code: RCA2530
dealer price: 39.45
list price: 80.50


model: RCA2535, product code: RCA2535
dealer price: 39.45
list price: 80.50


model: RJA2515, product code: RJA2515
dealer price: 52.60
list price: 107.35


model: RJA2520, product code: RJA2520
dealer price: 52.60
list price: 107.35


model: RJA2525, product code: RJA2525
dealer price: 52.60
list price: 107.35


model: RJA2530, product code: RJA2530
dealer price: 52.60
list price: 107.35


model: RJA2535, product code: RJA2535
dealer price: 52.60
list price: 107.35


model: RCA1015, product code: RCA1015
dealer price: 16.51
list price: 33.70


model: RCA1020, product code: RCA1020
dealer price: 16.51
list price: 33.70


model: RCA1025, product code: RCA1025
dealer price: 16.51
l

In [12]:
# save dataframe to CSV file
df.to_csv(output_file, index=False)
print('Results saved to CSV file\n')

print('Results:')
print(df)

print(f'Number of products found: {len(df)}\n')
print(f'Number of products not found: {products_not_found}\n')
if products_not_found > 0:
    print('Products not found:')
    for product in products_not_found_list:
        print(f'\t{product}')

# close browser
driver.close()
driver.quit()

Results saved to CSV file

Results:
    Product Model Product Code Product Cost Product List Price
0         RCA2515      RCA2515        39.45              80.50
1         RCA2520      RCA2520        39.45              80.50
2         RCA2525      RCA2525        39.45              80.50
3         RCA2530      RCA2530        39.45              80.50
4         RCA2535      RCA2535        39.45              80.50
..            ...          ...          ...                ...
133       RKA0330      RKA0330         9.65              19.70
134       REA0315      REA0315        10.46              21.35
135       REA0320      REA0320        10.46              21.35
136       REA0325      REA0325        10.46              21.35
137       REA0330      REA0330        10.46              21.35

[138 rows x 4 columns]
Number of products found: 138

Number of products not found: 0

